In [1]:
import numpy as np
from numpy import random
from gurobipy import *

In [30]:
c = np.array([10,1,1,1,10,1,1,1,10,15,15,15])
b = np.array([1,1,1])
A = np.array([[1,0,0,1,0,0,1,0,0,0,0,0],
             [0,1,0,0,1,0,0,1,0,0,0,0],
             [0,0,1,0,0,1,0,0,1,0,0,0]])
v = []

v.append(np.array([1,0,0,0,1,0,0,0,1,1,1,1]))
# v.append(np.array([0,1,0,1,0,0,0,0,1,1,1,1]))
# v.append(np.array([0,0,0,0,0,0,1,1,1,0,0,1]))
# v.append(np.array([0,0,0,1,1,1,0,0,0,0,1,0]))

In [14]:
def master(c,b,A,v):
    # Solving the restricted master problem
    RMP = Model('RMP')


    # Define Variables
    alpha = {}

    for i in range(len(v)):
        alpha[i] = RMP.addVar(name='alpha'+str(i))

    # Define Objective
    RMP.setObjective(quicksum(alpha[i]*quicksum(c[j]*v[i][j] for j in range(len(v[i]))) for i in range(len(v))))


    # Define Constraints
    Constr = {} # indicator constraints
    Constr['alpha'] = RMP.addConstr(quicksum(alpha[i] for i in alpha.keys()),GRB.EQUAL,1,name='Constr_alpha')
    for row in range(len(b)):
        Constr[row] = RMP.addConstr(quicksum(alpha[i]*quicksum(A[row][j]*v[i][j] for j in range(len(v[i]))) for i in range(len(v))),
                                    GRB.EQUAL,1,name='Constr'+str(row))
    #     =======================================================================================


    #Add constraints and variables to model
    RMP.update()
    # Initialize Model and Solver Settings NOTE: DO NOT EDIT THE SETTINGS IN THIS BLOCK UNLESS OTHERWISE NOTED IN THE EXERCISE
    RMP.setParam('TimeLimit', 900)
    RMP.setParam('DualReductions',0)
    # ufl.setParam('NodeLimit', 10) # 


    RMP.modelSense = GRB.MINIMIZE
    RMP.update()
    # Optimize Model (you should see output when running this cell)
    RMP.optimize()
    return [i.X for j,i in alpha.items()], RMP.objVal

In [15]:
def dual(c,b,A,v):
    dual = Model('dual')

    # Define Variables
    pi = {}
    for ind_i in range(3):
        pi[ind_i] = dual.addVar(lb=-GRB.INFINITY, name='pi'+str(ind_i))
    pi_0 = dual.addVar(lb=-GRB.INFINITY, name='pi_0')

    # Define Objective
    dual.setObjective(pi_0+pi[0]+pi[1]+pi[2])

    rhs = []
    for i in range(len(v)):
        rhs.append(sum(c[j]*v[i][j] for j in range(len(v[i]))))

    # Define Constraints
    Constr = {} # indicator constraints

    for i in range(len(v)):
        Constr[i] = dual.addConstr(quicksum(pi[row]*quicksum(A[row][j]*v[i][j] for j in range(len(v[i]))) for row in range(len(b)))+pi_0,
                                    '<=',rhs[i],name='Constr'+str(i))

    #     =======================================================================================


    #Add constraints and variables to model
    dual.update()
    # Initialize Model and Solver Settings NOTE: DO NOT EDIT THE SETTINGS IN THIS BLOCK UNLESS OTHERWISE NOTED IN THE EXERCISE
    dual.setParam('TimeLimit', 900)
    dual.setParam('DualReductions',0)
    # ufl.setParam('NodeLimit', 10) # 


    dual.modelSense = GRB.MAXIMIZE
    dual.update()
    # Optimize Model (you should see output when running this cell)
    dual.optimize()
    return np.array([pi[0].X,pi[1].X,pi[2].X]), pi_0.X, dual.objVal

In [22]:
def reduce(A,pis,mu):
    a = c-np.matmul(pis,A)-mu
    b = np.where(a<0)[0]
    k = [np.power(2,len(b))/2]
    while k[-1] > 1:
        k.append(k[-1]/2)
    for i in range(np.power(2,len(b))):
        d = np.array([0,0,0,0,0,0,0,0,0,0,0,0])
        for j in range(len(b)):
            if np.floor(i/k[j]) % 2 == 1:
                d[b[j]] = 1
        for j in range(3): 
            if d[j]==1: # if 
                d[9]=1
        for j in range(3,6): 
            if d[j]==1: # if 
                d[10]=1
        for j in range(6,9): 
            if d[j]==1: # if 
                d[11]=1
        print(d)
        v.append(d)

In [31]:
alpha, primal_obj = master(c,b,A,v)

Changed value of parameter TimeLimit to 900.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter DualReductions to 0
   Prev: 1  Min: 0  Max: 1  Default: 1
Optimize a model with 4 rows, 1 columns and 4 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 4 rows and 1 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.5000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  7.500000000e+01


In [32]:
pis, mu, dual_obj = dual(c,b,A,v)

Changed value of parameter TimeLimit to 900.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter DualReductions to 0
   Prev: 1  Min: 0  Max: 1  Default: 1
Optimize a model with 1 rows, 4 columns and 4 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+01, 8e+01]
Presolve time: 0.01s
Presolved: 1 rows, 4 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          0s
       1    7.5000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds
Optimal objective  7.500000000e+01


In [33]:
pis

array([75.,  0.,  0.])

In [35]:
reduce(A,np.array([25,25,25]),mu)

[0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 1 0 0 1]
[0 0 0 0 0 0 0 1 0 0 0 1]
[0 0 0 0 0 0 0 1 1 0 0 1]
[0 0 0 0 0 0 1 0 0 0 0 1]
[0 0 0 0 0 0 1 0 1 0 0 1]
[0 0 0 0 0 0 1 1 0 0 0 1]
[0 0 0 0 0 0 1 1 1 0 0 1]
[0 0 0 0 0 1 0 0 0 0 1 0]
[0 0 0 0 0 1 0 0 1 0 1 1]
[0 0 0 0 0 1 0 1 0 0 1 1]
[0 0 0 0 0 1 0 1 1 0 1 1]
[0 0 0 0 0 1 1 0 0 0 1 1]
[0 0 0 0 0 1 1 0 1 0 1 1]
[0 0 0 0 0 1 1 1 0 0 1 1]
[0 0 0 0 0 1 1 1 1 0 1 1]
[0 0 0 0 1 0 0 0 0 0 1 0]
[0 0 0 0 1 0 0 0 1 0 1 1]
[0 0 0 0 1 0 0 1 0 0 1 1]
[0 0 0 0 1 0 0 1 1 0 1 1]
[0 0 0 0 1 0 1 0 0 0 1 1]
[0 0 0 0 1 0 1 0 1 0 1 1]
[0 0 0 0 1 0 1 1 0 0 1 1]
[0 0 0 0 1 0 1 1 1 0 1 1]
[0 0 0 0 1 1 0 0 0 0 1 0]
[0 0 0 0 1 1 0 0 1 0 1 1]
[0 0 0 0 1 1 0 1 0 0 1 1]
[0 0 0 0 1 1 0 1 1 0 1 1]
[0 0 0 0 1 1 1 0 0 0 1 1]
[0 0 0 0 1 1 1 0 1 0 1 1]
[0 0 0 0 1 1 1 1 0 0 1 1]
[0 0 0 0 1 1 1 1 1 0 1 1]
[0 0 0 1 0 0 0 0 0 0 1 0]
[0 0 0 1 0 0 0 0 1 0 1 1]
[0 0 0 1 0 0 0 1 0 0 1 1]
[0 0 0 1 0 0 0 1 1 0 1 1]
[0 0 0 1 0 0 1 0 0 0 1 1]
[0 0 0 1 0 0 1 0 1 0 1 1]
[0 0 0 1 0 0

In [36]:
alpha, primal_obj = master(c,b,A,v)

Changed value of parameter TimeLimit to 900.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter DualReductions to 0
   Prev: 1  Min: 0  Max: 1  Default: 1
Optimize a model with 4 rows, 513 columns and 1860 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [2e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.01s
Presolved: 4 rows, 513 columns, 1860 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.6000000e+01   5.000000e-01   0.000000e+00      0s
       4    2.5500000e+01   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.02 seconds
Optimal objective  2.550000000e+01


In [37]:
pis, mu, dual_obj = dual(c,b,A,v)

Changed value of parameter TimeLimit to 900.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter DualReductions to 0
   Prev: 1  Min: 0  Max: 1  Default: 1
Optimize a model with 513 rows, 4 columns and 1860 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 8e+01]
Presolve removed 450 rows and 0 columns
Presolve time: 0.01s
Presolved: 63 rows, 4 columns, 207 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          0s
       5    2.5500000e+01   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.01 seconds
Optimal objective  2.550000000e+01


In [40]:
reduce(A,np.array([25,25,25]),mu)

[0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 1 0 0 1]
[0 0 0 0 0 0 0 1 0 0 0 1]
[0 0 0 0 0 0 0 1 1 0 0 1]
[0 0 0 0 0 0 1 0 0 0 0 1]
[0 0 0 0 0 0 1 0 1 0 0 1]
[0 0 0 0 0 0 1 1 0 0 0 1]
[0 0 0 0 0 0 1 1 1 0 0 1]
[0 0 0 0 0 1 0 0 0 0 1 0]
[0 0 0 0 0 1 0 0 1 0 1 1]
[0 0 0 0 0 1 0 1 0 0 1 1]
[0 0 0 0 0 1 0 1 1 0 1 1]
[0 0 0 0 0 1 1 0 0 0 1 1]
[0 0 0 0 0 1 1 0 1 0 1 1]
[0 0 0 0 0 1 1 1 0 0 1 1]
[0 0 0 0 0 1 1 1 1 0 1 1]
[0 0 0 0 1 0 0 0 0 0 1 0]
[0 0 0 0 1 0 0 0 1 0 1 1]
[0 0 0 0 1 0 0 1 0 0 1 1]
[0 0 0 0 1 0 0 1 1 0 1 1]
[0 0 0 0 1 0 1 0 0 0 1 1]
[0 0 0 0 1 0 1 0 1 0 1 1]
[0 0 0 0 1 0 1 1 0 0 1 1]
[0 0 0 0 1 0 1 1 1 0 1 1]
[0 0 0 0 1 1 0 0 0 0 1 0]
[0 0 0 0 1 1 0 0 1 0 1 1]
[0 0 0 0 1 1 0 1 0 0 1 1]
[0 0 0 0 1 1 0 1 1 0 1 1]
[0 0 0 0 1 1 1 0 0 0 1 1]
[0 0 0 0 1 1 1 0 1 0 1 1]
[0 0 0 0 1 1 1 1 0 0 1 1]
[0 0 0 0 1 1 1 1 1 0 1 1]
[0 0 0 1 0 0 0 0 0 0 1 0]
[0 0 0 1 0 0 0 0 1 0 1 1]
[0 0 0 1 0 0 0 1 0 0 1 1]
[0 0 0 1 0 0 0 1 1 0 1 1]
[0 0 0 1 0 0 1 0 0 0 1 1]
[0 0 0 1 0 0 1 0 1 0 1 1]
[0 0 0 1 0 0

[1 1 1 0 1 1 0 1 0 1 1 1]
[1 1 1 0 1 1 0 1 1 1 1 1]
[1 1 1 0 1 1 1 0 0 1 1 1]
[1 1 1 0 1 1 1 0 1 1 1 1]
[1 1 1 0 1 1 1 1 0 1 1 1]
[1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 1 0 0 0 0 0 1 1 0]
[1 1 1 1 0 0 0 0 1 1 1 1]
[1 1 1 1 0 0 0 1 0 1 1 1]
[1 1 1 1 0 0 0 1 1 1 1 1]
[1 1 1 1 0 0 1 0 0 1 1 1]
[1 1 1 1 0 0 1 0 1 1 1 1]
[1 1 1 1 0 0 1 1 0 1 1 1]
[1 1 1 1 0 0 1 1 1 1 1 1]
[1 1 1 1 0 1 0 0 0 1 1 0]
[1 1 1 1 0 1 0 0 1 1 1 1]
[1 1 1 1 0 1 0 1 0 1 1 1]
[1 1 1 1 0 1 0 1 1 1 1 1]
[1 1 1 1 0 1 1 0 0 1 1 1]
[1 1 1 1 0 1 1 0 1 1 1 1]
[1 1 1 1 0 1 1 1 0 1 1 1]
[1 1 1 1 0 1 1 1 1 1 1 1]
[1 1 1 1 1 0 0 0 0 1 1 0]
[1 1 1 1 1 0 0 0 1 1 1 1]
[1 1 1 1 1 0 0 1 0 1 1 1]
[1 1 1 1 1 0 0 1 1 1 1 1]
[1 1 1 1 1 0 1 0 0 1 1 1]
[1 1 1 1 1 0 1 0 1 1 1 1]
[1 1 1 1 1 0 1 1 0 1 1 1]
[1 1 1 1 1 0 1 1 1 1 1 1]
[1 1 1 1 1 1 0 0 0 1 1 0]
[1 1 1 1 1 1 0 0 1 1 1 1]
[1 1 1 1 1 1 0 1 0 1 1 1]
[1 1 1 1 1 1 0 1 1 1 1 1]
[1 1 1 1 1 1 1 0 0 1 1 1]
[1 1 1 1 1 1 1 0 1 1 1 1]
[1 1 1 1 1 1 1 1 0 1 1 1]
[1 1 1 1 1 1 1 1 1 1 1 1]


In [41]:
alpha, primal_obj = master(c,b,A,v)

Changed value of parameter TimeLimit to 900.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter DualReductions to 0
   Prev: 1  Min: 0  Max: 1  Default: 1
Optimize a model with 4 rows, 1025 columns and 3716 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [2e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.01s
Presolved: 4 rows, 1025 columns, 3716 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.6000000e+01   5.000000e-01   0.000000e+00      0s
       4    2.5500000e+01   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds
Optimal objective  2.550000000e+01


In [42]:
pis, mu, dual_obj = dual(c,b,A,v)

Changed value of parameter TimeLimit to 900.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter DualReductions to 0
   Prev: 1  Min: 0  Max: 1  Default: 1
Optimize a model with 1025 rows, 4 columns and 3716 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 8e+01]
Presolve removed 962 rows and 0 columns
Presolve time: 0.01s
Presolved: 63 rows, 4 columns, 207 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          0s
       8    2.5500000e+01   0.000000e+00   0.000000e+00      0s

Solved in 8 iterations and 0.01 seconds
Optimal objective  2.550000000e+01


In [45]:
pis

array([8.5, 8.5, 8.5])